# Collecting socioeconomic and electoral data

### Prerequisites

In [1]:
# Libraries

import pandas as pd # data wrangling
import numpy as np # math operations
import math # math operations
import os # directories
import time # system time
import random # random number generation
import pickle # data compression
import re # regular expressions
import unidecode # regular expressions

import urllib.request # scraping
import requests # scraping
from bs4 import BeautifulSoup # scraping
import ctypes # interface to C
import tweepy # twitter 

import sys # system limit (preventing infinite running)
sys.setrecursionlimit(100000)

import selenium # chrome driver
from selenium import webdriver # chrome driver
import selenium.common.exceptions as selexcept # exception handling

### Socioeconomic data

In [2]:
# Retrieve socioeconomic data from URL, save as df and discard first 6 empty rows

url_strukturdaten = "https://www.bundeswahlleiter.de/dam/jcr/f7566722-a528-4b18-bea3-ea419371e300/btw17_strukturdaten.csv"

socioeconomic_df = pd.read_csv(
    url_strukturdaten, encoding="ISO-8859-1", 
    delimiter=';', 
    decimal=",")
socioeconomic_df.columns = socioeconomic_df.iloc[7]
socioeconomic_df = socioeconomic_df.iloc[8:] 
socioeconomic_df = socioeconomic_df.reset_index(drop=True) 

In [3]:
# Drop redundant columns, NaN rows and rows with state-wide total votes

socioeconomic_df = socioeconomic_df.drop(socioeconomic_df.columns[[7, 8, 51]], axis=1)
socioeconomic_df['Wahlkreis-Nr.'] = socioeconomic_df['Wahlkreis-Nr.'].apply(lambda x: int(x)) 
mask1 = socioeconomic_df[socioeconomic_df['Wahlkreis-Nr.'] > 900].index
mask2 = socioeconomic_df[socioeconomic_df['Wahlkreis-Nr.'].isnull()].index
socioeconomic_df = socioeconomic_df.drop(mask1 | mask2)

C:\Users\wimme\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  import sys


In [4]:
socioeconomic_df.head()

7,Land,Wahlkreis-Nr.,Wahlkreis-Name,Gemeinden am 31.12.2015 (Anzahl),Fläche am 31.12.2015 (km²),Bevölkerung am 31.12.2015 - Insgesamt (in 1000),Bevölkerung am 31.12.2015 - Deutsche (in 1000),Zu- (+) bzw. Abnahme (-) der Bevölkerung 2015 - Geburtensaldo (je 1000 Einwohner),Zu- (+) bzw. Abnahme (-) der Bevölkerung 2015 - Wanderungssaldo (je 1000 Einwohner),Alter von ... bis ... Jahren am 31.12.2015 - unter 18 (%),...,Sozialversicherungspflichtig Beschäftigte am 30.06.2016 - Öffentliche und private Dienstleister (%),Sozialversicherungspflichtig Beschäftigte am 30.06.2016 - Übrige Dienstleister und 'ohne Angabe' (%),Empfänger(innen) von Leistungen nach SGB II am 31.12.2016 - insgesamt (je 1000 Einwohner),Empfänger(innen) von Leistungen nach SGB II am 31.12.2016 - nicht erwerbsfähige Hilfebedürftige (%),Empfänger(innen) von Leistungen nach SGB II am 31.12.2016 - Ausländer (%),Arbeitslosenquote März 2017 - insgesamt,Arbeitslosenquote März 2017 - Männer,Arbeitslosenquote März 2017 - Frauen,Arbeitslosenquote März 2017 - 15 bis unter 20 Jahre,Arbeitslosenquote März 2017 - 55 bis unter 65 Jahre
0,Schleswig-Holstein,1,Flensburg  Schleswig,130,"2128,1","282,8","266,7","-3,7","12,3","16,5",...,"16,4","35,2","88,5","26,3","18,8","7,2","8,2","6,2","4,2","7,3"
1,Schleswig-Holstein,2,Nordfriesland  Dithmarschen Nord,197,2777,"232,3","219,7",-5,"13,2","16,1",...,"12,4","31,7","70,9","26,2","16,7","7,2","7,7","6,6","3,6","8,4"
2,Schleswig-Holstein,3,Steinburg  Dithmarschen Süd,178,"2000,5","220,8","209,8","-5,3","11,9","16,7",...,"15,9","30,3","80,1",26,"21,1","6,6","7,2",6,"5,2","6,6"
3,Schleswig-Holstein,4,Rendsburg-Eckernförde,163,"2164,8","248,7","239,4","-3,4",10,"17,3",...,"15,5","34,7","59,4","28,1","23,8","5,1","5,5","4,7","3,2","5,6"
4,Schleswig-Holstein,5,Kiel,3,143,268,"242,9","-0,1","11,9","14,6",...,"25,2","38,6","138,7","26,4","28,3","8,8",10,"7,6","6,1","8,7"


In [5]:
new_colnames = [
    'bundesland', 
    'wahlkreis_nr', 
    'wahlkreis', 
    'n_municipalities', 
    'area_sqkm',
    'pop_k', 
    'pop_german_k', 
    'net_births_per_k', 
    'net_migration_per_k', 
    'share_u18', 
    'share_0_18', 
    'share_25_34', 
    'share_35_59', 
    'share_60_74', 
    'share_75_inf', 
    'share_pop_nomigration',
    'share_pop_migration', 
    'share_catholic', 
    'share_protestant',
    'share_other_religion', 
    'share_homeowners', 
    'newly_built_appartments_per_k', 
    'appartments_per_k', 
    'household_income_per_capita', 
    'bip_per_capita',
    'cars_per_k',
    'graduates_berufsschule_per_k',
    'graduates_allgemeinbildend_per_k',
    'share_no_graduation',
    'share_hauptschulabschluss',
    'share_mittlere_reife',
    'share_fachhochschulreife', 
    'children_childcare_per_k',
    'businesses_per_k',
    'craft_businesses_per_k',
    'workforce_per_k',
    'share_workforce_primary_sector',
    'share_workforce_industrial_sector',
    'share_workforce_trade_hospiality',
    'share_workforce_service_public_private',
    'share_workforce_service_misc',
    'welfare_recipients_per_k',
    'share_welfare_recipients_unemployable',
    'share_welfare_recipients_foreign',
    'unemployment_rate',
    'share_unemployment_rate_female',
    'share_unemployment_rate_male',
    'share_unemployment_rate_15_19',
    'share_unemployment_rate_55_64'
]

In [6]:
# Rename columns

socioeconomic_df.columns = new_colnames

In [8]:
# Convert columns w/ numbers to numeric

non_numeric_columns = [0, 1, 2]
numeric_columns = [
    element for element in range(len(socioeconomic_df.columns)) if element not in non_numeric_columns]
for i in numeric_columns:
    socioeconomic_df[socioeconomic_df.columns[i]] = \
    socioeconomic_df[socioeconomic_df.columns[i]].apply(
        lambda x: x.replace(',','.')).astype('float', errors = 'ignore')

# note: " - " is represented as square for column Wahlkreis-Name, coded as " \x96 "; 
# need to address if column is to be used

### Election results (2017 federal elections)

In [9]:
url_zweitstimmen = "https://www.bundeswahlleiter.de/dam/jcr/72f186bb-aa56-47d3-b24c-6a46f5de22d0/btw17_kerg.csv"

In [10]:
zweitstimmen_df = pd.read_csv(
    url_zweitstimmen, 
    encoding="ISO-8859-1", 
    delimiter=';', 
    skiprows=5)

In [11]:
# Drop redundant rows and columns

zweitstimmen_df = zweitstimmen_df.iloc[2:]
zweitstimmen_df = zweitstimmen_df.reset_index(drop=True)
relevant_columns = [0, 1, 2, 17, 21, 25, 29, 33, 37, 41, 45] 
zweitstimmen_df = zweitstimmen_df.iloc[:, relevant_columns]

In [12]:
# Rename columns

new_colnames = [
    'wahlkreis_nr', 
    'district', 
    'bundesland_nr', 
    'zweitstimmen_total', 
    'cdu', 
    'spd', 
    'linke', 
    'gruene', 
    'csu', 
    'fdp', 
    'afd'
]

zweitstimmen_df.columns = new_colnames

In [13]:
numeric_columns = [
    'wahlkreis_nr', 
    'bundesland_nr', 
    'zweitstimmen_total', 
    'cdu', 
    'spd', 
    'linke', 
    'gruene', 
    'csu', 
    'fdp', 
    'afd']

for i in numeric_columns:
    zweitstimmen_df[i] = pd.to_numeric(zweitstimmen_df[i], errors = 'coerce')

In [14]:
# Filter out NaN-rows and rows with state-wide total votes

mask1 = zweitstimmen_df[zweitstimmen_df['bundesland_nr'] == 99.0].index
mask2 = zweitstimmen_df[zweitstimmen_df['bundesland_nr'].isnull()].index
zweitstimmen_df = zweitstimmen_df.drop(mask1 | mask2)

C:\Users\wimme\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  """


In [15]:
# Merge CDU and CSU votes

zweitstimmen_df['cdu_csu'] = np.where(
    zweitstimmen_df['csu'].isnull(), 
    zweitstimmen_df['cdu'], 
    zweitstimmen_df['csu'])
zweitstimmen_df = zweitstimmen_df.drop(columns = ['cdu', 'csu'])

In [16]:
# Convert votes to shares

zweitstimmen_df['cdu_csu'] = zweitstimmen_df['cdu_csu'] / zweitstimmen_df['zweitstimmen_total']
zweitstimmen_df['spd'] = zweitstimmen_df['spd'] / zweitstimmen_df['zweitstimmen_total']
zweitstimmen_df['linke'] = zweitstimmen_df['linke'] / zweitstimmen_df['zweitstimmen_total']
zweitstimmen_df['gruene'] = zweitstimmen_df['gruene'] / zweitstimmen_df['zweitstimmen_total']
zweitstimmen_df['fdp'] = zweitstimmen_df['fdp'] / zweitstimmen_df['zweitstimmen_total']
zweitstimmen_df['afd'] = zweitstimmen_df['afd'] / zweitstimmen_df['zweitstimmen_total']

In [17]:
zweitstimmen_df.head()

,wahlkreis_nr,district,bundesland_nr,zweitstimmen_total,spd,linke,gruene,fdp,afd,cdu_csu
0,1.0,Flensburg â Schleswig,1.0,170465.0,0.236928,0.082140,0.130842,0.111196,0.068360,0.342123
1,2.0,Nordfriesland â Dithmarschen Nord,1.0,138071.0,0.225391,0.062207,0.109683,0.130730,0.065401,0.383339
2,3.0,Steinburg â Dithmarschen SÃ¼d,1.0,130878.0,0.227357,0.066719,0.099024,0.132169,0.085423,0.361910
3,4.0,Rendsburg-EckernfÃ¶rde,1.0,156267.0,0.228877,0.063750,0.123743,0.122041,0.074091,0.362105
4,5.0,Kiel,1.0,152069.0,0.238102,0.102230,0.171915,0.117078,0.069074,0.267878


### Merging socioeconomic and election data

In [18]:
socioeconomics_zweitstimmen_df = socioeconomic_df.join(
    zweitstimmen_df.set_index('wahlkreis_nr'), 
    on='wahlkreis_nr')

In [19]:
socioeconomics_zweitstimmen_df.head()

,bundesland,wahlkreis_nr,wahlkreis,n_municipalities,area_sqkm,pop_k,pop_german_k,net_births_per_k,net_migration_per_k,share_u18,...,share_unemployment_rate_55_64,district,bundesland_nr,zweitstimmen_total,spd,linke,gruene,fdp,afd,cdu_csu
0,Schleswig-Holstein,1,Flensburg  Schleswig,130.0,2128.1,282.8,266.7,-3.7,12.3,16.5,...,7.3,Flensburg â Schleswig,1.0,170465.0,0.236928,0.082140,0.130842,0.111196,0.068360,0.342123
1,Schleswig-Holstein,2,Nordfriesland  Dithmarschen Nord,197.0,2777.0,232.3,219.7,-5.0,13.2,16.1,...,8.4,Nordfriesland â Dithmarschen Nord,1.0,138071.0,0.225391,0.062207,0.109683,0.130730,0.065401,0.383339
2,Schleswig-Holstein,3,Steinburg  Dithmarschen Süd,178.0,2000.5,220.8,209.8,-5.3,11.9,16.7,...,6.6,Steinburg â Dithmarschen SÃ¼d,1.0,130878.0,0.227357,0.066719,0.099024,0.132169,0.085423,0.361910
3,Schleswig-Holstein,4,Rendsburg-Eckernförde,163.0,2164.8,248.7,239.4,-3.4,10.0,17.3,...,5.6,Rendsburg-EckernfÃ¶rde,1.0,156267.0,0.228877,0.063750,0.123743,0.122041,0.074091,0.362105
4,Schleswig-Holstein,5,Kiel,3.0,143.0,268.0,242.9,-0.1,11.9,14.6,...,8.7,Kiel,1.0,152069.0,0.238102,0.102230,0.171915,0.117078,0.069074,0.267878


In [20]:
socioeconomics_zweitstimmen_df.columns

Index(['bundesland', 'wahlkreis_nr', 'wahlkreis', 'n_municipalities',
       'area_sqkm', 'pop_k', 'pop_german_k', 'net_births_per_k',
       'net_migration_per_k', 'share_u18', 'share_0_18', 'share_25_34',
       'share_35_59', 'share_60_74', 'share_75_inf', 'share_pop_nomigration',
       'share_pop_migration', 'share_catholic', 'share_protestant',
       'share_other_religion', 'share_homeowners',
       'newly_built_appartments_per_k', 'appartments_per_k',
       'household_income_per_capita', 'bip_per_capita', 'cars_per_k',
       'graduates_berufsschule_per_k', 'graduates_allgemeinbildend_per_k',
       'share_no_graduation', 'share_hauptschulabschluss',
       'share_mittlere_reife', 'share_fachhochschulreife',
       'children_childcare_per_k', 'businesses_per_k',
       'craft_businesses_per_k', 'workforce_per_k',
       'share_workforce_primary_sector', 'share_workforce_industrial_sector',
       'share_workforce_trade_hospiality',
       'share_workforce_service_public_priva

In [67]:
# Save output

with open('../3_output/socioeconomics_zweitstimmen.pickle', 'wb') as handle:
    pickle.dump(socioeconomics_zweitstimmen_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [68]:
socioeconomics_zweitstimmen_df.to_csv(
    '../3_output/socioeconomics_zweitstimmen_df.csv', index=False, encoding='utf-8-sig')